In [0]:
import os
import pandas as pd
from rdflib import Graph

# Chemin Databricks
RDF_PATH = "../data/chicago_rdf"
OUTPUT_PATH = "../data/chicago_csv"


In [0]:
from rdflib import Graph
import pandas as pd

# Charger le RDF
RDF_FILES = [
    os.path.join(RDF_PATH, f)
    for f in os.listdir(RDF_PATH)
    if f.endswith(".rdf")
]


In [0]:

g = Graph()
g.parse(RDF_FILES[0], format="xml")

print("Nombre de triplets :", len(g))


In [0]:
predicates = set()

for s, p, o in g:
    predicates.add(p)

for p in list(predicates)[:20]:
    print(p)

In [0]:
import os
import pandas as pd
from rdflib import Graph, URIRef

# Namespace des champs utiles
NS_DAILY = {
    "route": URIRef("https://data.cityofchicago.org/resource/jyb9-n7fm/route"),
    "date": URIRef("https://data.cityofchicago.org/resource/jyb9-n7fm/date"),
    "day_type": URIRef("https://data.cityofchicago.org/resource/jyb9-n7fm/daytype"),
    "ridership": URIRef("https://data.cityofchicago.org/resource/jyb9-n7fm/rides")
}

os.makedirs(OUTPUT_PATH, exist_ok=True)

all_dfs = []

for rdf_file in RDF_FILES:
    print(f"Traitement de {os.path.basename(rdf_file)}")

    g = Graph()
    g.parse(rdf_file, format="xml")

    records = {}

    for s, p, o in g:
        if p in NS_DAILY.values():
            if s not in records:
                records[s] = {
                    "date": None,
                    "route": None,
                    "day_type": None,
                    "ridership": None
                }

            if p == NS_DAILY["date"]:
                records[s]["date"] = str(o)

            elif p == NS_DAILY["route"]:
                records[s]["route"] = str(o)

            elif p == NS_DAILY["day_type"]:
                records[s]["day_type"] = str(o)

            elif p == NS_DAILY["ridership"]:
                try:
                    records[s]["ridership"] = int(o)
                except:
                    records[s]["ridership"] = None

    df = pd.DataFrame.from_dict(records, orient="index")

    # Nettoyage
    df["date"] = pd.to_datetime(df["date"], errors="coerce")
    df = df.dropna(subset=["date", "route", "ridership"])
    df["route"] = df["route"].str.upper().str.strip()
    df["day_type"] = df["day_type"].str.capitalize()

    all_dfs.append(df)

# Concatenation finale
df_daily_all = pd.concat(all_dfs, ignore_index=True)

# Export CSV
OUTPUT_FILE = os.path.join(
    OUTPUT_PATH,
    "CTA_Ridership_Daily_by_Route.csv"
)

df_daily_all.to_csv(OUTPUT_FILE, index=False)

print(f"CSV GLOBAL généré : {OUTPUT_FILE}")
print(df_daily_all.head())


In [0]:
df_daily_all.shape